In [49]:
import os
import re
end =re.compile(r'\bEND=(-?\d+)\D')
svlen =re.compile(r'\bSVLEN=(-?\d+)\D')


In [117]:

def line_parser(line):
    end =re.compile(r'\bEND=(-?\d+)\D')
    svlen =re.compile(r'\bSVLEN=(-?\d+)\D')
    lines = line.strip().split('\t')
    CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT, *gt_list = lines
    END = end.findall(INFO)[0]
    SVLEN = svlen.findall(INFO)[0]
    return int(END), int(SVLEN), CHROM, int(POS),ID,REF,ALT,QUAL,FILTER,INFO,FORMAT, gt_list

cutoff=0.5

def combine_gt(gt_list, svlen, gt_list2, svlen2):
    if len(gt_list)!= len(gt_list2):
        print("Error: gt_list and gt_list2 have different length")
    c = []
    for i in range(len(gt_list)):
        if gt_list[i] == "0|0:NA:NA:NA:NA" and gt_list2[i] == "0|0:NA:NA:NA:NA":
            c.append("0|0:NA:NA:NA:NA")
        else:
            if gt_list[i] == "0|0:NA:NA:NA:NA":
                c.append(gt_list2[i])
            elif gt_list2[i] == "0|0:NA:NA:NA:NA":
                c.append(gt_list[i])
            elif svlen2 > svlen:
                c.append(gt_list2[i])
            else:
                c.append(gt_list[i])
    # print(c)
    return c

def combine_line(f, baseline, pass_line, cutoff = 0.5):
    try:
        line =  next(f)
    except:
        pass_line.append(baseline.strip())
        return pass_line
    END, SVLEN, CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT, gt_list = line_parser(baseline)
    END2, SVLEN2, CHROM2,POS2,ID2,REF2,ALT2,QUAL2,FILTER2,INFO2,FORMAT2, gt_list2 = line_parser(line)
    if POS2 >= END or CHROM2 != CHROM or ALT2 != ALT:
        pass_line.append(baseline.strip())
        baseline = line
    elif POS <= POS2 < END:
        if END2 <= END:
            overlap_length = END2 - POS2
            
        elif END <= END2:
            overlap_length = END - POS2
            
        if (cutoff*int(SVLEN2) <= overlap_length) and (cutoff*int(SVLEN) <= overlap_length):
            maxEND = max(END, END2)
            maxSVLEN = max(END, END2) - POS
            new_gt_list = combine_gt(gt_list, SVLEN, gt_list2, SVLEN2)
            new_info = "END=" + str(maxEND) + ";SVLEN=" + str(maxSVLEN) + ";SVTYPE=" + ALT
            new_line = '\t'.join([CHROM, str(POS), ID, REF, ALT, QUAL, FILTER, new_info, FORMAT, '\t'.join(new_gt_list)])
            # pass_line.append(new_line)
            baseline = new_line
        else:
            pass_line.append(baseline.strip())
            baseline = line
    return combine_line(f, baseline, pass_line, cutoff = 0.5)


with open('./tmp_20.vcf', 'r') as f:
    pass_line = []
    for line in f:
        if line.startswith('#'):
            pass_line.append(line.strip())
        else:
            baseline = line.strip()
            break
    pass_line = combine_line(f, baseline, pass_line, cutoff = 0.5)
        
print(len(pass_line))

with open('./tmp_20_combine.vcf', 'w') as f:
    for line in pass_line:
        print(line, file=f)

17


In [18]:
def combine_line(f, baseline, pass_line):
    try:
        line =  next(f)
    except:
        return pass_line
    END, SVLEN, CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT, gt_list = line_parser(baseline)
    END2, SVLEN2, CHROM2,POS2,ID2,REF2,ALT2,QUAL2,FILTER2,INFO2,FORMAT2, gt_list2 = line_parser(line)
    if POS2 >= END or CHROM2 != CHROM or ALT2 != ALT:
        pass_line.append(baseline)
        baseline = line
    elif POS <= POS2 < END:
        if END2 <= END:
            overlap_length = END2 - POS2
            
        elif END <= END2:
            overlap_length = END - POS2
            
        if (cutoff*int(SVLEN2) <= overlap_length) and (cutoff*int(SVLEN) <= overlap_length):
            maxEND = max(END, END2)
            maxSVLEN = max(END, END2) - POS
            new_gt_list = combine_gt(gt_list, gt_list2)
            new_info = "END=" + str(maxEND) + ";SVLEN=" + str(maxSVLEN) + ";SVTYPE=" + ALT
            new_line = '\t'.join([CHROM, str(POS), ID, REF, ALT, QUAL, FILTER, new_info, FORMAT, '\t'.join(new_gt_list)])
            # pass_line.append(new_line)
            baseline = new_line
    return combine_line(f, baseline, pass_line)

In [ ]:
# 定义两个列表
a = ['str1', 'str2', 'NA', 'str4', 'str5']
b = ['str1', 'NA', 'str3', 'str4', 'NA']
c = []
# 合并两个列表并去除NA
for i in range(len(a)):
    if a[i] == 'NA' and b[i] == 'NA':
        c.append('NA')
    else:
        if a[i] == 'NA':
            c.append(b[i])
        elif b[i] == 'NA':
            c.append(a[i])

# 输出合并后的列表
print(combined)


In [15]:
def line_conf(a, b):
    def line(x):
        return a * x + b
    return line

In [136]:
a = " <DEL> "
# b =a.lstrip("<").rstrip(">")
b= None
b = a.strip().strip("<>")
b

'DEL'

In [46]:
def test(f , line, list):
    try:
        line = next(f)
        list.append(line)
    except:
        return list
    return test(f, line, list)

with open("tmp.txt", "r") as f:
    a = []
    baseline = next(f)
    tmp = test(f, baseline, a)
tmp

['2 6\n', '4 10\n']